In [2]:
import os
print("Current Working Directory:", os.getcwd())

Current Working Directory: d:\chest_cancer_classification_project\research


In [3]:
%pwd

'd:\\chest_cancer_classification_project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\chest_cancer_classification_project'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/sleepy05/chest_cancer_classification_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="sleepy05"
os.environ["MLFLOW_TRACKING_PASSWORD"]="36d91d8a2e8d96ee3f8bcb8f5fbd4c33ec51b249"

In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/sleepy05/chest_cancer_classification_project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score() 

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-06-02 00:48:29,657: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-02 00:48:29,664: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-02 00:48:29,664: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 17s 2s/step - loss: 0.1903 - accuracy: 0.9706
[2025-06-02 00:48:47,418: INFO: common: json file saved at: scores.json]


2025/06/02 00:48:49 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-06-02 00:48:51,018: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\anshi\AppData\Local\Temp\tmplvqommm3\model\data\model\assets
[2025-06-02 00:48:51,553: INFO: builder_impl: Assets written to: C:\Users\anshi\AppData\Local\Temp\tmplvqommm3\model\data\model\assets]


2025/06/02 00:49:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


[2025-06-02 00:54:11,430: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None)': /sleepy05/chest_cancer_classification_project.mlflow/api/2.0/mlflow-artifacts/artifacts/06051988f0594f5aa2212f21f58ee54f/f9816848976143c28987fc399213ebc9/artifacts/model/data/model/variables/variables.data-00000-of-00001]
[2025-06-02 01:03:39,109: WARNING: connectionpool: Retrying (Retry(total=3, connect=5, read=3, redirect=5, status=5)) after connection broken by 'ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)': /sleepy05/chest_cancer_classification_project.mlflow/api/2.0/mlflow-artifacts/artifacts/06051988f0594f5aa2212f21f58ee54f/f9816848976143c28987fc399213ebc9/artifacts/model/data/model/variables/variables.data-00000-of-00001]


Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/06/02 01:11:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 7
Created version '7' of model 'VGG16Model'.
2025/06/02 01:11:48 INFO mlflow.tracking._tracking_service.client: 🏃 View run incongruous-kit-442 at: https://dagshub.com/sleepy05/chest_cancer_classification_project.mlflow/#/experiments/0/runs/f9816848976143c28987fc399213ebc9.
2025/06/02 01:11:48 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sleepy05/chest_cancer_classification_project.mlflow/#/experiments/0.
